<a href="https://colab.research.google.com/github/ThomazSIUFLA/CourseManager/blob/master/KLABIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Nome da pasta referente ao passo a passo
p_a_p_1 = '1_passo_a_passo/'

# Caminho da pasta com os arquivos
path = '/content/drive/MyDrive/KLABIN/'

# Nome dos arquivos
path_estradas = 'Estradas.shp' # estradas
path_outras_areas = 'Outras_Areas.shp' # outras áreas
path_infraestrutura = 'Infraestrutura.shp' # infraestrutura
path_area_p = 'Area_Produtiva.shp' # área produtiva
path_veg = 'Vegetacao.shp' # vegetação
path_app = 'APP.shp' # app
path_mde = 'Declividade_DSR.tif' # MDE


# Selecionando dos arquivos apenas aqueles que entrarão como área produtiva
# Dos arquivos lidos acima, somente algumas feições dentro dos shapes entrarão como área produtiva, nessa parte é realizada a seleção

# Cria uma lista para cada arquivo com as codificações dos tipos que entrarão como área produtiva
list_estradas = [2]
list_oareas = [13, 11, 16, 17, 18, 30, 31, 32]
list_infra = [0, 12]


# Configurando drive
# Essa parte habilita o drive para que possamos trabalhar com arquivos presentes no mesmo
from google.colab import drive
drive.mount('/content/drive')

# Configurando ambiente para processamento dos dados
!pip install wheel pandas shapely gdal fiona pyproj six rtree geopandas rasterio

# Importando bibliotecas necessárias
import gdal
import os
import ogr
import geopandas as gpd
import pandas as pd
import numpy as np


# Lendo arquivos necessários
estradas = gpd.read_file(path + path_estradas) # estradas
outras_areas = gpd.read_file(path + path_outras_areas) # outras áreas
infraestrutura = gpd.read_file(path + path_infraestrutura) # infraestrutura
area_p = gpd.read_file(path + path_area_p) # área produtiva
veg = gpd.read_file(path + path_veg) # vegetação
app = gpd.read_file(path + path_app) # app


# Seleciona somente as feições produtivas (de acordo dados inseridos na lista)
estr_prod = estradas[estradas['TIPO'].isin(list_estradas)]
oareas_prod = outras_areas[outras_areas['TIPO'].isin(list_oareas)]
infra_prod = infraestrutura[infraestrutura['TIPO'].isin(list_infra)]

# Unindo arquivos
# União das feições dos arquivos que entrarão como área produtiva
areap_oareas = gpd.overlay(area_p, oareas_prod, how='union', make_valid = False)
areap_oareas_infra = gpd.overlay(areap_oareas, infra_prod, how='union', make_valid = False)


# Dissolve seguido de buffer de 10m nas áreas unidas acima
areap_oareas_infra_buff_dis = areap_oareas_infra.dissolve()
# A ferramenta buffer faz com que o novo arquivo se transforme em GeoSerie ao invés de GeoDataFrame
areap_oareas_infra_buff_dis['geometry'] = areap_oareas_infra_buff_dis.geometry.buffer(10)


# Cortando as estradas apartir do dissolve/buffer
estr_prod_clip = gpd.clip(estr_prod, areap_oareas_infra_buff_dis)

# Ajeitando colunas
# Alterando o nome das colunas
areap_oareas_infra = areap_oareas_infra.loc[:, ~areap_oareas_infra.columns.str.endswith('_2')]
areap_oareas_infra = areap_oareas_infra.loc[:, ~areap_oareas_infra.columns.str.endswith('_1')]

# Removendo colunas duplicadas
areap_oareas_infra = areap_oareas_infra.loc[:,~areap_oareas_infra.columns.duplicated()]

# Excluindo colunas indesejadas
areap_oareas_infra = areap_oareas_infra.drop(areap_oareas_infra.filter(regex='created').columns, axis=1)
areap_oareas_infra = areap_oareas_infra.drop(areap_oareas_infra.filter(regex='last_edi').columns, axis=1)
areap_oareas_infra = areap_oareas_infra.drop(areap_oareas_infra.filter(regex='SHAPE_S').columns, axis=1)
areap_oareas_infra = areap_oareas_infra.drop(areap_oareas_infra.filter(regex='Global').columns, axis=1)

# Unindo as estradas após o clip com o merge das demais áreas
area_prod = gpd.overlay(areap_oareas_infra, estr_prod_clip, how='union', make_valid = False)

# Dissolve
# Dissolvendo o arquivo com a área produtiva
area_prod_dis = area_prod.dissolve()


# Tirando espaços dentro do dissolve
# Quando o dissolve é feito restam alguns "buracos", essa parte corrigiremos os buracos
# Criando um polígono convexo que contenha todos os pontos do shape dissolvido
poli = area_prod_dis['geometry'].convex_hull
poli = gpd.GeoDataFrame(poli, crs = area_p.crs) # transforma em geodataframe o arquivo do explode
poli.columns = ['geometry']

# Diferença entre o dissolve e o poligono gerado
buracos = poli.difference(area_prod_dis, align=True)
buracos = gpd.GeoDataFrame(buracos, crs = area_p.crs)
buracos.columns = ['geometry']


# Multiparte para single parte (para eliminar polígonos indesejados)
area_prod_dis_single = buracos.geometry.explode()

# Excluindo polígono de fora
area_prod_dis_single = gpd.GeoDataFrame(area_prod_dis_single, crs = area_p.crs) # transforma em geodataframe o arquivo do explode
area_prod_dis_single.columns = ['geometry'] # da nome pra coluna
area_prod_dis_single['area'] = area_prod_dis_single.geometry.area # calcula a área
area_prod_dis_single = area_prod_dis_single.loc[area_prod_dis_single['area'] != area_prod_dis_single['area'].max()] # descarta a feição de maior área


# Unido arquivo acima com o dissolve
area_prod = gpd.overlay(area_prod_dis, area_prod_dis_single, how='union', make_valid = False)


# Faz dissolve novamente com o arquivo da última união
area_prod = area_prod.dissolve()


# Interseção entre os arquivos dissolve e vegetação, para retirar do arquivo as áreas com vegetação, essas não entrarão como área produtiva
area_prod_inter = gpd.overlay(area_prod, veg, how ='intersection', make_valid = False)
area_prod_inter = area_prod_inter.loc[:,~area_prod_inter.columns.duplicated()] # remove colunas duplicadas


# Faz a diferença simétrica do arquivo da interseção e do dissolve
a_prod_parcial = gpd.overlay(area_prod_inter, area_prod, how = 'symmetric_difference', make_valid = False)


# Seleciona feições não especificadas nas listas
estr_n_prod = estradas[~estradas['TIPO'].isin(list_estradas)]
oareas_n_prod = outras_areas[~outras_areas['TIPO'].isin(list_oareas)]
infra_n_prod = infraestrutura[~infraestrutura['TIPO'].isin(list_infra)]


# Unindo áreas que não entrarão como área produtiva
app_veg = gpd.overlay(app, veg, how='union', make_valid = False) # app e vegetação
estr_oareas_n_prod = gpd.overlay(estr_n_prod, oareas_n_prod, how='union', make_valid = False) # estradas e outras áreas
estr_oareas_n_prod_app_veg = gpd.overlay(app_veg, estr_oareas_n_prod, how='union', make_valid = False) # app, vegetação, estradas e outras áreas
area_n_p = gpd.overlay(estr_oareas_n_prod_app_veg, infra_n_prod, how='union', make_valid = False) # app, vegetação, estradas, outras áreas e infraestrutura




In [ ]:
# Removendo áreas unidas acima
area_prod_final = a_prod_parcial.difference(area_n_p, align=True)
# A ferramenta acima faz com que o novo arquivo se transforme em GeoSerie ao invés de GeoDataFrame
# Transforma o arquvo em GeoDataFrame
area_prod_final = gpd.GeoDataFrame(area_prod_final, crs = area_p.crs)
area_prod_final.columns = ['geometry']
area_prod_final = area_prod_final[~area_prod_final['geometry'].isnull()]
area_prod_final.to_file(path + p_a_p_1 + 'area_prod_final.shp') # salva arquivo de área produtiva final

# Gera um buffer de 4.5m no shape de estradas não produtivas
estr_n_prod_buff = estr_n_prod
estr_n_prod_buff['geometry'] = estr_n_prod_buff.geometry.buffer(4.5)
estr_n_prod_buff.to_file(path + p_a_p_1 + 'estr_n_prod_buff.shp') # salva o arquivo na pasta

# Calcula a declividade a partir do MDE
os.system('gdaldem slope -compute_edges '+str(path)+str(path_mde)+' '+str(path)+str(p_a_p_1)+str('slope.tif')+'')

/usr/local/lib/python3.7/dist-packages/geopandas/base.py:31: UserWarning: The indices of the two GeoSeries are different.
  warn("The indices of the two GeoSeries are different.")
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  """
/usr/local/lib/python3.7/dist-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


0

In [ ]:
# Recorta o raster de declividade utilizando o buffer gerado anteriormente como máscara
os.system('gdalwarp -of GTiff -cutline /content/drive/MyDrive/KLABIN/1_passo_a_passo/area_prod_final.shp -cl area_prod_final -crop_to_cutline /content/drive/MyDrive/KLABIN/1_passo_a_passo/slope.tif /content/drive/MyDrive/KLABIN/1_passo_a_passo/clip.tif')


256

In [ ]:

area_prod_final.to_file(path + p_a_p_1 + 'area_prod_final.shp')

In [ ]:
area_prod_final.to_file(path + p_a_p_1 + 'area_prod_final.shp')

In [ ]:
area_prod_final['geometry'].isnull().values.any()

True

In [ ]:
# Recorta o raster de declividade utilizando o buffer gerado anteriormente como máscara
os.system('gdalwarp -of GTiff -cutline /content/drive/MyDrive/KLABIN/1_passo_a_passo/area_prod_final.shp -cl area_prod_final -crop_to_cutline /content/drive/MyDrive/KLABIN/1_passo_a_passo/slope.tif /content/drive/MyDrive/KLABIN/1_passo_a_passo/clip.tif')


256

In [ ]:
gdal_translate -projwin 541101.0362 7245990.6843 548809.8174 7238167.6405 -a_nodata 0.0 -of GTiff C:/Users/ZETTA/Downloads/slope.tif C:/Users/ZETTA/AppData/Local/Temp/processing_qiBlag/6a96ffd1aaff45a795fcf3414986b589/OUTPUT.tif



# Recorta o raster de declividade utilizando o buffer gerado anteriormente como máscara
os.system('gdalwarp -of GTiff -cutline /content/drive/MyDrive/KLABIN/1_passo_a_passo/area_p_buffer.shp -cl area_p_buffer -crop_to_cutline /content/drive/MyDrive/KLABIN/1_passo_a_passo/slope.tif /content/drive/MyDrive/KLABIN/1_passo_a_passo/clip.tif')
# Grava geometrias vetoriais da estrada no raster recortado no passo anterior
os.system('gdal_rasterize -l estrada_buffer -burn 999.0 /content/drive/MyDrive/KLABIN/estrada_buffer.shp /content/drive/MyDrive/KLABIN/slope.tif')

In [ ]:
# Nome da pasta referente ao passo a passo
p_a_p_2 = '2_passo_a_passo/'

# Caminho da pasta com os arquivos
path = '/content/drive/MyDrive/KLABIN/'

# Nome dos arquivos
path_estradas = 'Estradas.shp' # estradas
path_outras_areas = 'Outras_Areas.shp' # outras áreas
path_infraestrutura = 'Infraestrutura.shp' # infraestrutura
path_area_p = 'Area_Produtiva.shp' # área produtiva
path_mde = 'Declividade_DSR.tif' # MDE


# Selecionando dos arquivos apenas aqueles que entrarão como área produtiva
# Dos arquivos lidos acima, somente algumas feições dentro dos shapes entrarão como área produtiva, nessa parte é realizada a seleção

# Cria uma lista para cada arquivo com as codificações dos tipos que entrarão como área produtiva
list_estradas = [2]
list_oareas = [13, 11, 16, 17, 18, 30, 31, 32]
list_infra = [0, 12]


# Configurando drive
# Essa parte habilita o drive para que possamos trabalhar com arquivos presentes no mesmo
from google.colab import drive
drive.mount('/content/drive')

# Configurando ambiente para processamento dos dados
!pip install wheel pandas shapely gdal fiona pyproj six rtree geopandas rasterio

# Importando bibliotecas necessárias
import gdal
import os
import ogr
import geopandas as gpd
import pandas as pd
import numpy as np


# Lendo arquivos necessários
estradas = gpd.read_file(path + path_estradas) # estradas
outras_areas = gpd.read_file(path + path_outras_areas) # outras áreas
infraestrutura = gpd.read_file(path + path_infraestrutura) # infraestrutura
area_p = gpd.read_file(path + path_area_p) # área produtiva


# Seleciona somente as feições produtivas (de acordo dados inseridos na lista)
estr_prod = estradas[estradas['TIPO'].isin(list_estradas)]
oareas_prod = outras_areas[outras_areas['TIPO'].isin(list_oareas)]
infra_prod = infraestrutura[infraestrutura['TIPO'].isin(list_infra)]

# Unindo arquivos
# União das feições dos arquivos que entrarão como área produtiva
areap_oareas = gpd.overlay(area_p, oareas_prod, how='union', make_valid = False)
areap_oareas_infra = gpd.overlay(areap_oareas, infra_prod, how='union', make_valid = False)
areap_oareas_infra.reindex(areap_oareas_infra.columns, axis = 'columns')

# Dissolve seguido de buffer de 10m nas áreas unidas acima
areap_oareas_infra_buff_dis = areap_oareas_infra.dissolve().buffer(10)
# A ferramenta buffer faz com que o novo arquivo se transforme em GeoSerie ao invés de GeoDataFrame
#areap_oareas_infra_buff_dis = areap_oareas_infra_dis
#areap_oareas_infra_buff_dis['geometry'] = areap_oareas_infra_dis.geometry.buffer(10)


# Cortando as estradas apartir do dissolve/buffer
estr_prod_clip = gpd.clip(estr_prod, areap_oareas_infra_buff_dis)

# Ajeitando colunas
# Alterando o nome das colunas
areap_oareas_infra = areap_oareas_infra.loc[:, ~areap_oareas_infra.columns.str.endswith('_2')]
areap_oareas_infra = areap_oareas_infra.loc[:, ~areap_oareas_infra.columns.str.endswith('_1')]

# Removendo colunas duplicadas
areap_oareas_infra = areap_oareas_infra.loc[:,~areap_oareas_infra.columns.duplicated()]

# Excluindo colunas indesejadas
areap_oareas_infra = areap_oareas_infra.drop(areap_oareas_infra.filter(regex='created').columns, axis=1)
areap_oareas_infra = areap_oareas_infra.drop(areap_oareas_infra.filter(regex='last_edi').columns, axis=1)
areap_oareas_infra = areap_oareas_infra.drop(areap_oareas_infra.filter(regex='SHAPE_S').columns, axis=1)
areap_oareas_infra = areap_oareas_infra.drop(areap_oareas_infra.filter(regex='Global').columns, axis=1)

# Unindo as estradas após o clip com o merge das demais áreas
area_prod = gpd.overlay(areap_oareas_infra, estr_prod_clip, how='union')
#area_prod = areap_oareas_infra_estr.dissolve()

# Criando coluna 'N' com valor 1
area_prod['N'] = 1

# Salva o arquivo
area_prod.to_file(path + p_a_p_2 +'area_prod.shp')

gdal.DEMProcessing((path + p_a_p_2 + 'slope.tif'), srcDS = (path + path_mde), format = 'GTiff', processing = 'slope')

src = gdal.Open(path + p_a_p_2 + 'slope.tif')
xmin, xres, xskew, ymax, yskew, yres  = src.GetGeoTransform()
xmax = xmin + (src.RasterXSize * xres)
ymin = ymax + (src.RasterYSize * yres)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:55: UserWarning: `keep_geom_type=True` in overlay resulted in 353 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:56: UserWarning: `keep_geom_type=True` in overlay resulted in 417 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: UserWarning: `keep_geom_type=True` in overlay resulted in 2441 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.


In [ ]:
# Rasterização do shape para uniformização da área
gdal.RasterizeLayer((path + p_a_p_2 + 'area_prod.tif'), (path + p_a_p_2 +'area_prod.shp'), format = 'GTiff', xRes = 12.5, yRes = 12.5, outputBounds = [str(xmin), str(ymin), str(xmax), str(ymax)], noData = 0)


<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7efe8170df90> >

In [ ]:
# Rasterização do shape para uniformização da área
os.system('gdal_rasterize -l area_prod -tr 12.5 12.5 -te '+str(xmin)+' '+str(ymin)+' '+str(xmax)+' '+str(ymax)+' -a N -a_nodata 999 -ot Float32 -of GTiff /content/drive/MyDrive/KLABIN/2_passo_a_passo/area_prod.shp /content/drive/MyDrive/KLABIN/2_passo_a_passo/area_prod.tif')
# Vetorização do arquivo raster 
#os.system('gdal_polygonize.py /content/drive/MyDrive/KLABIN/all_prod.tif /content/drive/MyDrive/KLABIN/all_prod.shp -f "ESRI Shapefile"')



0

In [ ]:
# Calcula a declividade a partir do MDE
os.system('gdaldem slope -compute_edges /content/drive/MyDrive/KLABIN/Declividade_DSR.tif /content/drive/MyDrive/KLABIN/slope.tif')

0

In [ ]:
src = gdal.Open('/content/drive/MyDrive/KLABIN/slope.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()
print(ulx)
print(uly)

lrx = ulx + (src.RasterXSize * xres)
lry = uly + (src.RasterYSize * yres)
print(lrx)
print(lry)

541042.3124998639
7246102.7500034515
548892.3124998639
7238102.7500034515


In [ ]:
# Transforma o arquivo raster em ASCII
os.system('gdal_translate -of GTiff /content/drive/MyDrive/KLABIN/slope.tif /content/drive/MyDrive/KLABIN/decliv_texto.asc')

0

In [ ]:
os.system('gdal_calc.py -A /content/drive/MyDrive/KLABIN/slope.tif --outfile=/content/drive/MyDrive/KLABIN/slope_class.tif --calc="(A<=37)*1+(A>37)*(A<=45)*2+(A>45)*3"')

0

In [ ]:
os.system('gdal_reclassify.py /content/drive/MyDrive/KLABIN/slope.tif /content/drive/MyDrive/KLABIN/slope_class2.tif -c "<37,<45,<100" -r "1,2,3" -d 0 -n true -p "COMPRESS=LZW"')

32512

In [ ]:
# Calcula a declividade a partir do MDE
os.system('gdaldem slope -compute_edges /content/drive/MyDrive/KLABIN/Declividade_DSR.tif /content/drive/MyDrive/KLABIN/slope.tif')
# olhar o tamanho da janela, testar diferentes tamanhos de janelas
#0 a 37, 45, acima de 45
#reclassify


# Gera um buffer de 4.5m no shape de estradas
estrada = gpd.read_file('/content/drive/MyDrive/KLABIN/Estradas.shp')
estrada_buffer = estrada.buffer(4.5)
estrada_buffer.to_file('/content/drive/MyDrive/KLABIN/estrada_buffer.shp')


# Grava geometrias vetoriais da estrada no raster recortado no passo anterior
os.system('gdal_rasterize -l estrada_buffer -burn 999.0 /content/drive/MyDrive/KLABIN/estrada_buffer.shp /content/drive/MyDrive/KLABIN/slope.tif')

0

In [ ]:
# Gera um buffer de 10m no shape de áreas produtivas
area_p = gpd.read_file('/content/drive/MyDrive/KLABIN/Area_Produtiva.shp')
area_p_buffer = area_p.buffer(10)
area_p_buffer.to_file('/content/drive/MyDrive/KLABIN/area_p_buffer.shp')



In [ ]:
# Recorta o raster de declividade utilizando o buffer gerado anteriormente como máscara
os.system('gdalwarp -of GTiff -cutline /content/drive/MyDrive/KLABIN/area_p_buffer.shp -cl area_p_buffer -crop_to_cutline /content/drive/MyDrive/KLABIN/slope.tif /content/drive/MyDrive/KLABIN/clip.tif')

0

In [ ]:
# Grava geometrias vetoriais da estrada no raster recortado no passo anterior
os.system('gdal_rasterize -l estrada_buffer -burn 99.0 /content/drive/MyDrive/KLABIN/estrada_buffer.shp /content/drive/MyDrive/KLABIN/clip.tif')

0

In [ ]:
# Produz um raster de proximidade (distância) da declividade com as estradas

#inserir máscara
os.system('gdal_proximity.py -srcband 1 -distunits PIXEL -values 99 -nodata 0.0 -ot Float32 -use_input_nodata YES -of GTiff /content/drive/MyDrive/KLABIN/clip.tif /content/drive/MyDrive/KLABIN/dist.tif')

0

In [ ]:
#rasterize overwrite com estradas como 999